In [1]:
import numpy as np 
import pandas as pd

np.random.seed(42)

In [48]:
iter_df = pd.DataFrame({"OrderDate": pd.date_range(start="2025-01-01", end="2025-08-31").tolist()})
df = pd.DataFrame({
    "OrderNumber": pd.Series(dtype="str"), "OrderDate": pd.Series(dtype="datetime64[ns]"), "OrderMonth": pd.Series(dtype="int"), "OrderYear": pd.Series(dtype="int"), 
    "Amount": pd.Series(dtype="float"), "ShippingCost": pd.Series(dtype="float"), "COGS": pd.Series(dtype="float"), "Profit": pd.Series(dtype="float"),
    "Return": pd.Series(dtype="int"), "ProductLine": pd.Series(dtype="str"), "BottomLine": pd.Series(dtype="float"),
    "CumulativeSales": pd.Series(dtype="float"), "CumulativeBottomLine": pd.Series(dtype="float")
    })

for i in range(len(iter_df)):
    orderDate = pd.to_datetime(iter_df.OrderDate.iloc[i])

    join_df = pd.DataFrame()
    cnt_orders = np.random.randint(low=0, high=20)

    returns = [0,0,0,0,0,0,0,0,0,1] # 1 in 10 chance of return
    products = ["Clothing", "Beauty", "Electronics", "Home"]
    
    # for order in range(cnt_orders):
    join_df["OrderNumber"] = np.random.randint(low=10000, high=99999, size=cnt_orders)
    join_df["OrderDate"] = [orderDate for i in range(cnt_orders)]
    join_df["OrderMonth"] = pd.to_datetime(join_df.OrderDate).dt.month.to_numpy()
    join_df["OrderYear"] = pd.to_datetime(join_df.OrderDate).dt.year.to_numpy()
    join_df["Amount"] = np.round(np.random.uniform(low=5, high=100, size=cnt_orders), 2)
    join_df["ShippingCost"] = np.round(join_df["Amount"] * np.random.uniform(0.01, 0.10, cnt_orders), 2)
    join_df["COGS"] = np.round(join_df["Amount"] * np.random.uniform(0.15, 0.30, cnt_orders), 2)
    join_df["Profit"] = join_df.Amount - (join_df.ShippingCost + join_df.COGS)
    join_df["Return"] = np.random.choice(returns, size=cnt_orders)
    join_df["ProductLine"] = np.random.choice(products, size=cnt_orders)

    join_df["BottomLine"] = np.where(join_df.Return == 1, -1 * (join_df.ShippingCost + join_df.COGS), join_df.Profit) # If the item gets returned, costs still apply

    df = pd.concat([df, join_df])

df["CumulativeSales"] = df.groupby("OrderMonth")["Amount"].cumsum()
df["CumulativeBottomLine"] = df.groupby("OrderMonth")["BottomLine"].cumsum()

In [49]:
salesByMonth = df.groupby("OrderMonth")["Amount"].sum().shift(1).fillna(0)
ordersByMonth = df.groupby("OrderMonth")["OrderNumber"].count().shift(1).fillna(0)

kpiMetrics = pd.DataFrame({"SalesTarget": pd.Series(dtype="float"), "OrdersTarget": pd.Series(dtype="float")})
kpiMetrics["SalesTarget"] = salesByMonth
kpiMetrics["OrdersTarget"] = ordersByMonth

kpiMetrics = kpiMetrics.reset_index()
df = df.merge(kpiMetrics, how="left", on="OrderMonth")

In [50]:
df = df.reset_index(drop=True)
df

,OrderNumber,OrderDate,OrderMonth,OrderYear,Amount,ShippingCost,COGS,Profit,Return,ProductLine,BottomLine,CumulativeSales,CumulativeBottomLine,SalesTarget,OrdersTarget
0,67348,2025-01-01 00:00:00,1,2025,40.75,2.54,10.03,28.18,0,Beauty,28.18,40.75,28.18,0.00,0.0
1,54509,2025-01-01 00:00:00,1,2025,21.96,2.06,5.13,14.77,0,Electronics,14.77,62.71,42.95,0.00,0.0
2,23801,2025-01-01 00:00:00,1,2025,41.80,3.88,6.75,31.17,0,Clothing,31.17,104.51,74.12,0.00,0.0
3,37946,2025-01-01 00:00:00,1,2025,36.44,0.49,5.69,30.26,0,Clothing,30.26,140.95,104.38,0.00,0.0
4,86261,2025-01-01 00:00:00,1,2025,99.52,4.70,26.47,68.35,0,Clothing,68.35,240.47,172.73,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2244,99771,2025-08-30 00:00:00,8,2025,83.71,0.88,14.01,68.82,0,Clothing,68.82,15932.58,10021.89,16435.38,302.0
2245,25257,2025-08-30 00:00:00,8,2025,67.30,5.94,20.19,41.17,0,Home,41.17,15999.88,10063.06,16435.38,302.0
2246,37976,2025-08-31 00:00:00,8,2025,59.17,1.44,16.08,41.65,0,Electronics,41.65,16059.05,10104.71,16435.38,302.0
2247,48961,2025-08-31 00:00:00,8,2025,57.05,5.68,10.44,40.93,0,Electronics,40.93,16116.10,10145.64,16435.38,302.0


In [51]:
df.to_excel("SalesData.xlsx")